## Bedrock model integration with Langchain Agents 

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio on ml.t3.medium instance*


### Financial stock analyzer example  

Certain applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Agent interface enables such flexibility for these applications. An agent has availability to a range of resources and selects which ones to utilize based on the user input. Agents are capable of using multiple tools and utilizing the output of one tool as the input for the next.  

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

In this notebook, we will demonstrate the use `Zero-shot ReAct` agent based on [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) framework to select the appropriate tool based exclusively on the tool's description. It requires you provide the description of each tool. 

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](1_setup.ipynb) notebook. ⚠️ ⚠️ ⚠️


In [ ]:

import json
import os
import sys
import boto3
import re
from functools import partial

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock

from langchain.chat_models import BedrockChat
from typing import Optional, List, Any

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)



In [ ]:

model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens": 2000}
llm = BedrockChat(model_id=model_id, client=boto3_bedrock, model_kwargs=model_parameter, verbose=True)


## Part 1 - Prepare dataset in SQLite Database
First we will create some sample data for stock analysis in SQLite database. This will later serve as a tool for our agent to look up information about stocks.

In [ ]:

## Create Sample data for subsequent SQL query usage

stock_ticker_data=[ 
    {
        "symbol" : "PRAA",
        "name" : "PRA Group, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "AMZN",
        "name" : "Amazon.com, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "TSLA",
        "name" : "Tesla Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "PAAS",
        "name" : "Pan American Silver Corp.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "PAAC",
        "name" : "Proficient Alpha Acquisition Corp.",
        "currency" : "USD",
        "stockExchange" : "NasdaqCM",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "RYAAY",
        "name" : "Ryanair Holdings plc",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "MPAA",
        "name" : "Motorcar Parts of America, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "STAA",
        "name" : "STAAR Surgical Company",
        "currency" : "USD",
        "stockExchange" : "NasdaqGM",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "RBCAA",
        "name" : "Republic Bancorp, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "AABA",
        "name" : "Altaba Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"    
    }, 
    {
        "symbol" : "AAXJ",
        "name" : "iShares MSCI All Country Asia ex Japan ETF",
        "currency" : "USD",
        "stockExchange" : "NasdaqGM",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "ZNWAA",
        "name" : "Zion Oil & Gas, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGM",
        "exchangeShortName" : "NASDAQ"
    }
]


In [ ]:

#sqllite3 Python library that provides a lightweight disk-based database that doesn’t require a separate server process
import sqlite3

#Creating a function for Db connection
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except FileExistsError as e:
        print(e)

    return conn


In [ ]:

#Creating a function for table creation
def run_sql(conn, sql_query):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(sql_query)
    except RuntimeError as e:
        print(e)
    

In [ ]:

#Create a Database Connection
db_name = "stock_ticker_database.db"
conn = create_connection(db_name)


In [ ]:
drop_table_sql = """DROP TABLE IF EXISTS stock_ticker;"""

#Create Table Query
create_table_sql = """CREATE TABLE stock_ticker (
	symbol text PRIMARY KEY,
	name text NOT NULL,
	currency text,
	stockExchange text, 
    exchangeShortName text
);"""


In [ ]:

# Calling create table user defined function
if conn is not None:
    # drop existing table
    run_sql(conn, drop_table_sql)
    
    # create projects table
    run_sql(conn, create_table_sql)
else:
    print("Error! cannot create the database connection.")

    

In [ ]:

#Insert the data created in the previous step. 
def insert_data(data):
    for item in data:
        conn.execute("INSERT INTO stock_ticker (symbol, name, currency,stockExchange, exchangeShortName ) VALUES (?, ?, ?, ?,?)", 
                    (item["symbol"], item["name"], item["currency"], item["stockExchange"],item["exchangeShortName"]))
    conn.commit()
    conn.close()
    

In [ ]:

# uncomment to insert data if the data doesn't exist in the table.
insert_data(stock_ticker_data)


## Part 2 - Database Tools
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintaining the conversational interactions.

### SQL Database Tool
The [LangChain](https://python.langchain.com/docs/get_started/introduction) includes tools to help build application that interact with relational databases. For details, read this document: https://python.langchain.com/docs/integrations/toolkits/sql_database

In the example below we will construct a workflow or a chain in LangChain parlance that will look up a stock in the SQLite database. Specifically this chain will be comprised of 3 steps:
- Step 1: Generate a SQL query given a user question and the database schema
- Step 2: Execute the query against the database
- Step 3: Generate a response based on the query results

We will subsequently use this chain as a tool in our agent.


In [ ]:

from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter


In [ ]:
# create a langchain database instance
db = SQLDatabase.from_uri("sqlite:///stock_ticker_database.db")

The helper function below will help us extract specific text contained within an XML tag. LLMs have a tendency to generate extra text that is not relevant to the user query. To get around this we will instruct the model to output the SQL query into an XML tag and use the function below to extract the SQL query from the output.

In [ ]:
def extract_from_xml_tag(response:str, tag:str) -> str:
    
    """Extract the text from the specified XML tag in the response string."""
    
    tag_txt = re.search(rf'<{tag}>(.*?)</{tag}>', response, re.DOTALL)
    if tag_txt:
        return tag_txt.group(1)
    else:
        return ""

Next we create a prompt that will be used to instruct the model to generate the SQL query. LangChain will automatically construct the relevant chain based on the prompt and the schema information captured from the `db` object that was created earlier. We will use the [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/get_started/) to add an additional step to the chain that will extract the SQL query from the <sql> XML tag.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

sql_template = """You are a ANSI SQL expert with access to a database with the following tables: 

<schema>
{table_info}
</schema>

Generate a SQL Query that retrieves the top {top_k} records. Do not explain the query, just output the SQL.
Place the output into <sql>...</sql> tags.

Question: {input}"""

sql_prompt = ChatPromptTemplate.from_template(sql_template)

query_writer_chain = create_sql_query_chain(llm, db, prompt=sql_prompt, k=100)

extract_sql = partial(extract_from_xml_tag, tag="sql")

query_writer_chain = query_writer_chain | RunnableLambda(extract_sql) # extract the SQL query from the response


In [ ]:
# Let's test if the query generation chain works
query_writer_chain.invoke({"question": "Retrieve the top 5 records from the stock_ticker table."})

Next we need to add a step to execute the generated SQL query. Fortunately LangChain already has a built-in [QuerySQLDataBaseTool](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.sql_database.tool.QuerySQLDataBaseTool.html) which does exactly this

In [ ]:
# let's test if the query works
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke({"query": "SELECT * FROM stock_ticker LIMIT 5;"})

The final step is to create a prompt that will take the output from the SQL Execution step and generate a response.

In [ ]:
answer_prompt = ChatPromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
<question> {question} </question>
<sql> 
{query} 
</sql>
<result>
{result}
</result>

Provide a response to the question. Do not provide any additional information, do not explain the SQL query, and do not say that the results came from a SQL query. 
The user has no knowledge of the backend data systems and mention of SQL will confuse them.
"""
)

answer_chain = answer_prompt | llm | StrOutputParser()

We can now combine the 3 steps into a single chain

In [ ]:
full_sql_chain = (
    RunnablePassthrough.assign(query=query_writer_chain).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_chain
)

In [ ]:

print(full_sql_chain.invoke({"question":"What is the ticker symbol for Tesla in stock ticker table?"}))


## Part 3 - Construct the tools for the ReACt agent
In this section we will define multiple functions that will act as tools that our agent can use to address the user's task. We will define the following tools:
- `get_stock_ticker`: This tool will look up a stock in the SQLite database using the SQL chain we defined earlier
- `get_stock_price`: This tool will look up the price of a stock ticker using an external API
- `get_stock_news`: This tool will look up the latest news about a stock ticker using an external API
- `get_financial_statements`: This tool will look up the financial statements of a stock ticker using an external API


In [ ]:
import json
import time
from bs4 import BeautifulSoup
import re
import requests

from langchain.agents import load_tools, AgentType, Tool, initialize_agent
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf

yf.pdr_override()
import pandas as pd
from datetime import datetime, timedelta

import warnings
import os

warnings.filterwarnings("ignore")
from langchain.tools import DuckDuckGoSearchRun
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain


def get_stock_ticker(query, db_chain=full_sql_chain):
    """
    Returns the ticker symbol and company name relevant to the given query.

    Args:
        query (str): The query to find the relevant ticker symbol and company name.
        db_chain (Chain, optional): The database chain to use. Defaults to full_sql_chain.

    Returns:
        tuple: A tuple containing the company name and ticker symbol.

    """

    response = db_chain.invoke(
        {
            "question": f"Return the ticker symbol and company name that is relevant to this query {query}. Place symbol into <symbol>...</symbol> tags and company name into <company>...</company> tags."
        }
    )
    symbol = extract_from_xml_tag(response, "symbol")
    company_name = extract_from_xml_tag(response, "company")

    return company_name, symbol


def get_stock_price(ticker, history=10):
    """
    Returns the stock data for the specified ticker symbol.

    Args:
        ticker(str): The ticker symbol for which to fetch the stock data.
        history(int): The number of days of historical data to fetch. Defaults to 10.

    Returns:
        tuple: A tuple containing the stock data and a unique name.

    """

    print(ticker)
    today = date.today()
    start_date = today - timedelta(days=history)
    data = pdr.get_data_yahoo(ticker, start=start_date, end=today)
    dataname = ticker + "_" + str(today)

    return data, dataname


# Fetch top 5 google news for given company name
def google_query(search_term):

    if "news" not in search_term:
        search_term = search_term + " stock news"
    url = f"https://www.google.com/search?q={search_term}"
    url = re.sub(r"\s", "+", url)
    return url


def get_recent_stock_news(company_name):
    """
    Fetches and returns the top 5 recent news articles for a given company from Google News.

    Args:
        company_name (str): The name of the company to fetch news for.

    Returns:
        str: A string containing the top 5 recent news articles for the company.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
    }

    g_query = google_query(company_name)
    res = requests.get(g_query, headers=headers).text
    soup = BeautifulSoup(res, "html.parser")
    news = []
    for n in soup.find_all("div", "n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div", "IJl0Z"):
        news.append(n.text)

    if len(news) > 6:
        news = news[:4]
    else:
        news = news
    news_string = ""
    for i, n in enumerate(news):
        news_string += f"{i}. {n}\n"
    top5_news = "Recent News:\n\n" + news_string

    return top5_news


def stock_news_search(company_name):
    search = DuckDuckGoSearchRun()
    return search("Stock news about " + company_name)


# Get financial statements from Yahoo Finance
def get_financial_statements(ticker):
    """
    Fetches and returns the balance sheet for a given company ticker from Yahoo Finance.

    Args:
        ticker (str): The ticker of the company to fetch the balance sheet for.

    Returns:
        str: A string representation of the company's balance sheet.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]
    else:
        ticker = ticker
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1] >= 3:
        balance_sheet = balance_sheet.iloc[:, :3]  # Only captures last 3 years of data
    balance_sheet = balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    return balance_sheet


Let's start by testing out some of these tools

In [ ]:
# Look up a stock ticker for a company given an arbitrary query
company_name, company_ticker = get_stock_ticker("What is the main business of Amazon?")
print("Company Name: ", company_name)
print("Company Ticker: ", company_ticker)


In [ ]:

print(get_stock_price("AMZN"))

In [ ]:

print(get_recent_stock_news("Amazon"))


In [ ]:

print(get_financial_statements("AMZN"))


In [ ]:

print(stock_news_search("Amazon"))


Before building an Agent, let's combine all of these tools into a static prompt to see how an LLM could make use of the information provided by these tools.

In [ ]:

def analyze_stock(query):
    
    """Construct a chain that will always invoke all of the tools and use the output to respond to the user query."""
    
    company_name, ticker = get_stock_ticker(query)
    print({"Query": query, "company_name": company_name, "Ticker": ticker})
    stock_data = get_stock_price(ticker, history=10)
    stock_financials = get_financial_statements(ticker)
    stock_news = get_recent_stock_news(company_name)

    analysis_template = """
    Given detail stock analysis, Use the available data and provide investment recommendation. 
    The user is fully aware about the investment risk, do not include any kind of warning like 
    'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer. 
    User question: {query}. You have the following information available about {company_name}. 
    Write (5-8) point investment analysis to answer user query, At the end conclude with proper explanation. 
    Try to Give positives and negatives: 
    <stock_price>
    {stock_data}
    </stock_price> 
    
   <stock_financials>
   {stock_financials}
   <stock_financials>
    
    
    <stock_news>
    {stock_news}
    </stock_news>
    
    Provide an analysis only base on the information provided above. Do not use any other external information or your own knowledge as the basis for the analysis.
    
    """

    analysis_prompt = ChatPromptTemplate.from_template(analysis_template)
    analysis_chain = analysis_prompt | llm | StrOutputParser()

    analysis = analysis_chain.invoke(
        {
            "query": query,
            "company_name": company_name,
            "stock_data": stock_data,
            "stock_financials": stock_financials,
            "stock_news": stock_news,
        }
    )

    return analysis

In [ ]:

analyze_result=analyze_stock("Is Amazon a good investment choice right now?")
print(analyze_result)


## Part 4 - Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In this section, rather than hardcoding the tools into a static chain, we will use the ReAct framework to select the appropriate tool based exclusively on the tool's description and the user input.

In [ ]:

# A few additional imports to build the agent
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool


When instantiating the ReAct agent, we will provide a list of tools and their descriptions. The agent will then select the appropriate tool based on the user input and the tool's description. We can provide this information by wrapping the tools in a `Tool` object.

In [ ]:

tools=[
    Tool(
        name="get_stock_ticker",
        func=get_stock_ticker,
        description="Get the company stock ticker"
    ),
    Tool(
        name="get_stock_price",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it "
    ),
    Tool(
        name="get_recent_stock_news",
        func=get_recent_stock_news,
        description="Use this to fetch recent news about stocks"
    ),

    Tool(
        name="get_financial_statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it"
    ) 


]


You may have noticed that in tool functions that we defined earlier, we wrote docstrings in the [Google-style format](https://google.github.io/styleguide/pyguide.html). To avoid duplication, we can use a pair of helper functions `extract_docstring_info` and `construct_format_tool_for_claude_prompt` to parse the docstring and convert it into a tool calling prompt format prescribed by the [Claude documentation](https://docs.anthropic.com/claude/docs/legacy-tool-use)

**Note:** The prompt based approach below for tool usage is now considered legacy. Anthropic introduced a json base format for tools as documented [here](https://docs.anthropic.com/claude/docs/tool-use-examples). This example will be updated once the Bedrock API and Langchain support the new format.

In [ ]:
from utils.prompt_utils import extract_docstring_info, construct_format_tool_for_claude_prompt

In [ ]:
tool_prompt = ""
for tool in tools:
    docstring_info = extract_docstring_info(tool.func.__doc__)
    tool_prompt += construct_format_tool_for_claude_prompt(tool.name, docstring_info['description'], docstring_info['params'])

print(tool_prompt)

We'll construct a prompt that will be used to instruct the model to select the appropriate tool based on the tool's description and the user input which will be injected based on the XML generated above. We are also providing placeholders for the chat history and the intermediate reasoning steps within the `{agent_scratchpad}`.

In [ ]:
agent_prompt_template = """You are a financial advisor who has access to a set of tools that can help answer questions about stocks and investments.

You have access to the following tools:

{tools}

You may invoke any tool like this:
<tool>$TOOL_NAME</tool><tool_input>$PARAMETER_VALUE</tool_input>
For example to use the tool 'get_stock_ticker', you would invoke it like this:
<tool>get_stock_ticker</tool>
<tool_input>"What is the main business of Amazon?"</tool_input>
<observation>Amazon.com, Inc., AMZN</observation>

When you are done, respond with a final answer between <final_answer></final_answer>. For example:

<final_answer>Investment Analysis of Amazon.com, Inc. (AMZN)</final_answer>

Make sure you make use of all the tools available to you and not bias your answer based on an output from a single tool or your own knowledge.

Begin!

Previous Conversation:
{chat_history}

Question: {input}
{agent_scratchpad}
"""

agent_prompt = ChatPromptTemplate.from_template(agent_prompt_template, partial_variables={"chat_history": "", "tools": tool_prompt})

In [ ]:
def convert_intermediate_steps(intermediate_steps):
    """
    Helper function to convert the intermediate reasoning steps to xml format that Claude is better able to understand.
    """
    
    log = ""
    for action, observation in intermediate_steps:

        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log

We can now combine the prompt, model, and the intermediate steps conversion function into a single chain.

In [ ]:
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import XMLAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | agent_prompt
    | llm.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

Lancgain provides a built-in [AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html) to construct the agent. This executor will essentially invoke the chain defined above until it generates the `<final_answer>` response.

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_response = agent_executor.invoke({"input": "Is Tesla a good investment choice right now?"})

In [ ]:
print(agent_response["output"])